In [84]:
#FOR ANALYSIS/CLEANING/COMPUTATION:
import pandas as pd
import numpy as np

#FOR VISUALIZATION:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [27]:
import time
import pandas as pd

print("Loading data file now, this could take a while depending on file size")
start = time.time()
df = pd.read_csv('submission.csv') # ADD-CSV
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds\n")
print("Number of rows : ",df.shape[0]," and the number of columns : ",df.shape[1])
missing_values = df.isna().sum().sum()
duplicated_values = df.duplicated().sum()
print(f'\nMissing values: {missing_values}')
print(f'Duplicated values: {duplicated_values}')
if missing_values >= 1:
    print('\nMissing values by column:')
    print(df.isna().sum())
print("\nUnique Values in Each Column:")
print(df.nunique())

Loading data file now, this could take a while depending on file size
Loading took 0.01 seconds

Number of rows :  2000  and the number of columns :  2

Missing values: 0
Duplicated values: 0

Unique Values in Each Column:
image_id    2000
caption     1801
dtype: int64


In [42]:
filter_df = df[df['caption'].str.contains('ค่ะ')]

In [43]:
filter_df

,image_id,caption,text_tokens
6,1735,มีตระกร้านั่งอยู่บนหญ้าค่ะ,มี ตระกร้า นั่ง อยู่ บน หญ้า
7,1440,มีแพะนั่งอยู่ในหญ้าใต้ต้นไม้ค่ะ,มี แพะ นั่ง อยู่ ใน หญ้า ใต้ ต้นไม้
16,978,มีชามใส่เนื้อสัตว์ ก๋วยเตี๋ยวและผักค่ะ,มี ชาม ใส่ เนื้อสัตว์ ก๋วยเตี๋ยว และ ผัก
31,1680,มีเห็ดอยู่บนโต๊ะพร้อมผงค่ะ,มี เห็ด อยู่ บน โต๊ะ พร้อม ผง
37,1577,มีสาขาเล็ก ๆ ที่มีเบอร์รี่เขียวอยู่ค่ะ,มี สาขา เล็ก ๆ ที่ มี เบอร์ รี่ เขียว อยู่
...,...,...,...
1944,168,มีปูอยู่บนพื้นใบค่ะ,มี ปู อยู่ บน พื้น ใบ
1960,336,มีแมลงเขียวนั่งอยู่บนหินค่ะ,มี แมลง เขียว นั่ง อยู่ บน หิน
1965,1433,มีอาหารจานนึงที่มีแซนวิชกับสลัดค่ะ,มี อาหาร จาน นึง ที่ มี แซ น วิ ช กับ สลัด
1985,984,มีสวิงอยู่กลางสนามค่ะ,มี สวิง อยู่ กลาง สนาม


In [4]:
import re
import numpy as np
def clean_text(text):
    if pd.isna(text):
        return np.nan
    text = re.sub(r'Ã[\x80-\xBF]+', ' ', text)
    text = re.sub(r'[^a-zA-Zก-ฮะ-์\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.lower()

In [5]:
df['Clean_text'] = df['caption'].apply(clean_text)

In [6]:
pd.set_option('display.max_colwidth', None)
print(df.iloc[4]['caption'])
print(df.iloc[4]['Clean_text'])

มีแมลงเล็ก ๆ สองตัวนั่งอยู่บนต้นไม้
มีแมลงเล็ก ๆ สองตัวนั่งอยู่บนต้นไม้


In [91]:
from pythainlp.corpus.common import thai_stopwords
thai_stopwords = list(thai_stopwords())

In [32]:
from pythainlp import word_tokenize
def text_process(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":", "!", '"', "ฯ",))
    final = word_tokenize(final , engine="newmm")
    final = " ".join(word for word in final)

    return final


In [33]:
df['text_tokens'] = df['caption'].apply(text_process)

In [35]:
custom_stopwords = {'ครับ', 'ค่ะ'}

In [36]:

def remove_stopwords(text):
    words = text.split()
    return ' '.join(word for word in words if word not in custom_stopwords)

In [38]:
df['text_tokens'] = df['text_tokens'].apply(remove_stopwords)

In [39]:
df

,image_id,caption,text_tokens
0,1354,มีแมลงตัวเล็กสีขาวอยู่บนทราย,มี แมลง ตัวเล็ก สี ขาว อยู่ บน ทราย
1,1413,มีกิ้งก่านั่งอยู่บนกิ่งไม้ในป่า,มี กิ้งก่า นั่ง อยู่ บน กิ่งไม้ ใน ป่า
2,1802,มีเรือที่วิ่งผ่านภูเขาที่มีบ้านอยู่,มี เรือ ที่ วิ่ง ผ่าน ภูเขา ที่ มี บ้าน อยู่
3,1243,มีลิงนั่งอยู่บนพื้นติดกับกําแพง,มี ลิง นั่ง อยู่ บน พื้น ติดกับ กํา แพง
4,693,มีแมลงเล็ก ๆ สองตัวนั่งอยู่บนต้นไม้,มี แมลง เล็ก ๆ สอง ตัว นั่ง อยู่ บน ต้นไม้
...,...,...,...
1995,29,มีสระน้ําเล็ก ๆ อยู่ตรงกลางเรือนกระจก,มี สระ น้ํา เล็ก ๆ อยู่ ตรงกลาง เรือนกระจก
1996,1065,มีดอกไม้สีแดงมากมายบนใบไม้เขียว,มี ดอกไม้ สีแดง มากมาย บน ใบไม้ เขียว
1997,195,ต้นไม้กําลังเติบโตอยู่ด้านข้างของถนนใกล้กับร่างกายของน้ํา,ต้นไม้ กํา ลัง เติบโต อยู่ ด้าน ข้าง ของ ถนน ใกล้ กับ ร่างกาย ของ น้ํา
1998,1026,มีดอกไม้เล็ก ๆ ที่เติบโตบนลําต้น,มี ดอกไม้ เล็ก ๆ ที่ เติบโต บน ลํา ต้น


In [44]:
from fixthaipdf import clean

In [54]:
df['Clean_text'] = df['text_tokens'].apply(clean)

In [55]:
df_filtered = df[df['text_tokens'] != df['Clean_text']]
df_filtered

,image_id,caption,text_tokens,Clean_text
0,1354,มีแมลงตัวเล็กสีขาวอยู่บนทราย,แมลง ตัวเล็ก สี ขาว อยู่ บน ทราย,แมลง ตัวเล็ก สีขาว อยู่บน ทราย
1,1413,มีกิ้งก่านั่งอยู่บนกิ่งไม้ในป่า,กิ้งก่า นั่ง อยู่ บน กิ่งไม้ ใน ป่า,กิ้งก่า นั่ง อยู่บน กิ่งไม้ ใน ป่า
2,1802,มีเรือที่วิ่งผ่านภูเขาที่มีบ้านอยู่,เรือ ที่ วิ่ง ผ่าน ภูเขา ที่ มี บ้าน อยู่,เรือ ที่วิ่ง ผ่าน ภูเขา ที่มีบ้าน อยู่
3,1243,มีลิงนั่งอยู่บนพื้นติดกับกําแพง,ลิง นั่ง อยู่ บน พื้น ติดกับ กํา แพง,ลิง นั่ง อยู่บน พื้น ติดกับ กำ แพง
4,693,มีแมลงเล็ก ๆ สองตัวนั่งอยู่บนต้นไม้,แมลง เล็ก ๆ สอง ตัว นั่ง อยู่ บน ต้นไม้,แมลง เล็ก ๆ สอง ตัว นั่ง อยู่บน ต้นไม้
...,...,...,...,...
1995,29,มีสระน้ําเล็ก ๆ อยู่ตรงกลางเรือนกระจก,สระ น้ํา เล็ก ๆ อยู่ ตรงกลาง เรือนกระจก,สระ น้ำ เล็ก ๆ อยู่ตรงกลาง เรือนกระจก
1996,1065,มีดอกไม้สีแดงมากมายบนใบไม้เขียว,ดอกไม้ สีแดง มากมาย บน ใบไม้ เขียว,ดอกไม้สีแดง มากมาย บน ใบไม้ เขียว
1997,195,ต้นไม้กําลังเติบโตอยู่ด้านข้างของถนนใกล้กับร่างกายของน้ํา,ต้นไม้ กํา ลัง เติบโต อยู่ ด้าน ข้าง ของ ถนน ใกล้ กับ ร่างกาย ของ น้ํา,ต้นไม้กำ ลัง เติบโต อยู่ด้าน ข้าง ของ ถนน ใกล้กับ ร่างกาย ของ น้ำ
1998,1026,มีดอกไม้เล็ก ๆ ที่เติบโตบนลําต้น,ดอกไม้ เล็ก ๆ ที่ เติบโต บน ลํา ต้น,ดอกไม้ เล็ก ๆ ที่ เติบโต บน ลำ ต้น


In [56]:
df['text_tokens'] = df['text_tokens'].apply(lambda x: x.replace(" ", ""))

In [57]:
df

,image_id,caption,text_tokens,Clean_text
0,1354,มีแมลงตัวเล็กสีขาวอยู่บนทราย,แมลงตัวเล็กสีขาวอยู่บนทราย,แมลง ตัวเล็ก สีขาว อยู่บน ทราย
1,1413,มีกิ้งก่านั่งอยู่บนกิ่งไม้ในป่า,กิ้งก่านั่งอยู่บนกิ่งไม้ในป่า,กิ้งก่า นั่ง อยู่บน กิ่งไม้ ใน ป่า
2,1802,มีเรือที่วิ่งผ่านภูเขาที่มีบ้านอยู่,เรือที่วิ่งผ่านภูเขาที่มีบ้านอยู่,เรือ ที่วิ่ง ผ่าน ภูเขา ที่มีบ้าน อยู่
3,1243,มีลิงนั่งอยู่บนพื้นติดกับกําแพง,ลิงนั่งอยู่บนพื้นติดกับกําแพง,ลิง นั่ง อยู่บน พื้น ติดกับ กำ แพง
4,693,มีแมลงเล็ก ๆ สองตัวนั่งอยู่บนต้นไม้,แมลงเล็กๆสองตัวนั่งอยู่บนต้นไม้,แมลง เล็ก ๆ สอง ตัว นั่ง อยู่บน ต้นไม้
...,...,...,...,...
1995,29,มีสระน้ําเล็ก ๆ อยู่ตรงกลางเรือนกระจก,สระน้ําเล็กๆอยู่ตรงกลางเรือนกระจก,สระ น้ำ เล็ก ๆ อยู่ตรงกลาง เรือนกระจก
1996,1065,มีดอกไม้สีแดงมากมายบนใบไม้เขียว,ดอกไม้สีแดงมากมายบนใบไม้เขียว,ดอกไม้สีแดง มากมาย บน ใบไม้ เขียว
1997,195,ต้นไม้กําลังเติบโตอยู่ด้านข้างของถนนใกล้กับร่างกายของน้ํา,ต้นไม้กําลังเติบโตอยู่ด้านข้างของถนนใกล้กับร่างกายของน้ํา,ต้นไม้กำ ลัง เติบโต อยู่ด้าน ข้าง ของ ถนน ใกล้กับ ร่างกาย ของ น้ำ
1998,1026,มีดอกไม้เล็ก ๆ ที่เติบโตบนลําต้น,ดอกไม้เล็กๆที่เติบโตบนลําต้น,ดอกไม้ เล็ก ๆ ที่ เติบโต บน ลำ ต้น


In [102]:
df = df.drop(columns=['caption', 'Clean_text'])
df = df.rename(columns={'text_tokens': 'caption'})

In [52]:
filter_df = df[df['text_tokens'].str.contains("มี")]

In [53]:
filter_df

,image_id,caption,text_tokens,Clean_text
2,1802,มีเรือที่วิ่งผ่านภูเขาที่มีบ้านอยู่,เรือ ที่ วิ่ง ผ่าน ภูเขา ที่ มี บ้าน อยู่,มี เรือ ที่วิ่ง ผ่าน ภูเขา ที่มีบ้าน อยู่
5,1695,มีผักเป็นจานบนโต๊ะที่มีมีด,ผัก เป็น จาน บน โต๊ะ ที่ มี มีด,มีผัก เป็น จาน บน โต๊ะ ที่มีมีด
12,149,มีผู้หญิงคนหนึ่งกําลังนั่งอยู่บนพื้นดินด้วยมีด,ผู้หญิง คน หนึ่ง กํา ลัง นั่ง อยู่ บน พื้นดิน ด้วย มีด,มีผู้หญิง คน หนึ่ง กำ ลัง นั่ง อยู่บน พื้นดิน ด้วย มีด
13,458,มีรูปปั้นทองขนาดใหญ่ในตึกที่มีพรมแดง,รูปปั้น ทอง ขนาดใหญ่ ใน ตึก ที่ มี พรม แดง,มีรูปปั้น ทอง ขนาดใหญ่ ใน ตึก ที่มีพรม แดง
19,1587,และทุ่งหญ้ามีภูเขาเป็นหลัก,และ ทุ่งหญ้า มี ภูเขา เป็นหลัก,และ ทุ่งหญ้า มีภูเขา เป็นหลัก
...,...,...,...,...
1984,1943,มีพืชที่มีใบสีเหลืองอยู่ตรงกลางของสนาม,พืช ที่ มี ใบ สีเหลือง อยู่ ตรงกลาง ของ สนาม,มีพืช ที่มี ใบ สีเหลือง อยู่ตรงกลาง ของ สนาม
1986,1199,มีช้อนสองช้อนบนจานที่มีช้อนสีน้ําเงินและส่วนที่เหลือช้อน,ช้อน สอง ช้อน บน จาน ที่ มี ช้อน สี น้ํา เงิน และ ส่วนที่เหลือ ช้อน,มีช้อน สอง ช้อน บน จาน ที่มีช้อน สีน้ำ เงิน และ ส่วนที่เหลือ ช้อน
1989,1162,มีตึกเล็กๆสีขาว มีหลังคาแดง และต้นปาล์ม,ตึก เล็ก ๆ สี ขาว มี หลังคา แดง และ ต้น ปาล์ม,มีตึก เล็ก ๆ สีขาว มีหลังคา แดง และ ต้น ปาล์ม
1992,542,ภูเขาในระยะทางที่มีรันเวย์และเมืองด้านล่าง,ภูเขา ใน ระยะทาง ที่ มี รันเวย์ และ เมือง ด้านล่าง,ภูเขา ใน ระยะทาง ที่มีรันเวย์ และ เมือง ด้านล่าง


In [58]:
df[df['image_id'] == 1]

,image_id,caption,text_tokens,Clean_text
1484,1,มีชามซุปกับอาหารทอดบนจานค่ะ,ชามซุปกับอาหารทอดบนจาน,ชาม ซุป กับ อาหาร ทอด บน จาน


In [51]:
df['text_tokens'] = df['text_tokens'].str.lstrip('มี')
# df['caption'] = df['caption'].str.lstrip('ใครบาง')

In [ ]:
# df.to_csv('submission_2d.csv', index=False)